# Partial Cognate Detection (Johann-Mattis List)

## 1 What is Partial Cognacy?

In a very general notion, cognacy is similar to the
concept of homology in biology ([Haggerty et al. 2014](:bib:Haggerty2014)), denoting a relation between words which
share a common history ([List 2014](:bib:List2014d)). In classical linguistics, borrowings are often excluded
from this notion ([Trask 2000](:bib:Trask2000)). Quantitative approaches additionally distinguish cognates which
have retained, and cognates which have shifted
their meaning (Starostin 2013b). Further aspects
of cognacy are rarely distinguished, although they
are obvious and common. Words which go back to
the same ancestor form can for example have been morphologically modified, such as French soleil
which does not go directly back to Latin sōl 'sun
but to *sōliculus* 'small sun' which is itself a derivation of *sōl* ([Meyer-Lübke 1911](:bib:Meyer-Luebke1911)).

Another problem are words which have
been created from two or more morphemes via
processes of compounding. While these cases
are rather rare in the core vocabulary of Indo-European languages, they are very frequent in
South-East Asian language families like Sino-Tibetan or Austro-Asiatic. As a typical example, compare the words for "sun" in the following table:

![img](img/s9-partial-cognates.png)

In 200 basic words
across 23 Chinese dialects (Ben Hamed and Wang
2006), for example, almost 50% of the nouns and
more than 30% of all words consist of two or more
morphemes, as illustrated in the following plot:

![img](img/s9-lens.jpg)

The presence of words consisting of more
than one morpheme challenges the notion that
words can either be cognate or not. It poses
problems for phylogenetic approaches which require binary presence-absence matrices as input
and model language evolution as cognate gain and
cognate loss ([Atkinson and Gray 2006](:bib:Atkinson2006)).

Partial cognacy is also a problem for current cognate detection algorithms which compare
words in their entirety (List 2014, [Turchin et al. 2010](:bib:Turchin2010)). Given the frequency of compound words
in South-East Asian languages, it is not surprising
that the algorithms perform much worse on diverse
South-East Asian language families, than they perform on other language families where compound-
ing is less frequent (List 2014:197f).



## 2 LingPy Algorithm for Partial Cognate Detection

The crucial idea behind the algorithm for partial cognate detection in LingPy is what could be called a "constrained-based partitioning". While the "normal" partitioning algorithms for cognate set partitioning do not work with constraints in any form, but simply derive a partition for a given set of input words from the pairwise distances between all words in a given meaning slot, the constrained-based partitioning adds specific constraints when constructing the distance matrix (or the network), based on the structure of the input data. If these constraints (called "filter rules" in the original paper by [List et al. 2016](:bib:List2016g)) are applied, the algorithm either refuses to draw a link between two nodes in the network corresponding to the matrix, or the distance score in the matrix is artificially set to its maximum of 1.0. The filtering rules themselves are necessary to account for certain properties of partial cognate detection, since we know well that certain conditions in which an unconstrained algorithm could erroneously cluster two words together, might otherwise not occur.

The partial cognate detection algorithm in LingPy is similar to the normal cognate detection algorithm. The only difference is that it does not cluster all words in a given meaning slot into partitions, but also all morphemes. That means that each word is represented by at least one, but potentially more than one morpheme, and that the morphemes constituting one word can be assigned to different cognate sets.

The filter rules that are currently used in LingPy's partial cognate detection algorithm are simple:

1. Don't accept matches between two or more morphemes occuring in the same word. 
2. Only accept one match between one morpheme in one word and another morpheme in another word.

Rule 1 essentially guarantees that the morphemes in one word are not assigned to the same partition. While this may sound surprising at first sight, given that we know well that certain processes like reduplication essentially replicate one morpheme inside the same word, it is necessary for practical reasons, since it would increase the search space and also create situations in which one word as a whole would be put into one cluster, which is then impossible to be aligned.

Rule 2 contradicts also with some theoretically well-known findings, but we need to accept it, since an n-to-n mapping between morphemes across words might again create in-alignable words, and -- as we shall see when looking at the way in which EDICTOR handles partial cognates -- it is of crucial importance for us to maintain that the partial cognates are still alignable, even if not in completely linear alignments.

Once the distance matrix or the similarity network is reconstructed on this basis (employing the filter rules or constraints), the same partitioning algorithm (e.g., UGPMA or Infomap) is applied as it was already used in the "normal" cognate detection task. The following figure illustrates this:

![img](img/s9-workflow.png)


## 3 Annotation of Partial Cognates in EDICTOR

### 3.1  Input Formats in EDICTOR

To understand how partial cognate annotation in EDICTOR works, it is crucial to understand in which format partial cognates are represented in EDICTOR. Partial cognates as well as partial alignments have, in contrast to "normal" cognates, the disadvantage of being no longer completely linar in their structure. For example, a compound word of structure "AB" may well be reflected as "BA" in another language (numerous examples for Chinese dialects exist, especially in cases like "she-dog" or "he-dog", with dialects differing in the order of "he/she" and "dog". 

Following our general annotation of cognates in LingPy and EDICTOR, to use numerical identifiers, and assuming that all words in the data are explicitly segmented into morphemes by adding the `+` character as a a secondary segmentation marker in the `TOKENS` column of an EDICTOR file, we can apply the same logic of numerical identifiers to all parts of a given compound (or otherwise multi-morphemic) word. For two words, like Chengdu `[` k o ŋ ⁴⁴ + k ə u ⁵³ `]` and Chaozhou `[` k a u ⁵³ + k a ŋ ³³ `]`, this means, that we assign the identifiers in the opposite order in a space-segmented list:

DOCULECT | TOKENS | COGIDS 
--- | --- | ---
Chengdu |  k o ŋ ⁴⁴ + k ə u ⁵³ | 1 2
Chaozhou | k a u ⁵³ + k a ŋ ³³ | 2 1
Hefei | k ə ŋ ²¹² + k ɯ ²⁴ | 1 2 

For our alignments, this also means that we store them in the opposite. For the given case, the alignment is almost trivial, as only one gap for Hefei is required:

DOCULECT | TOKENS | COGIDS | ALIGNMENT
--- | --- | --- | ---
Chengdu |  k o ŋ ⁴⁴ + k ə u ⁵³ | 1 2 | k o ŋ ⁴⁴ + k ə u ⁵³
Chaozhou | k a u ⁵³ + k a ŋ ³³ | 2 1 | k a u ⁵³ + k a ŋ ³³
Hefei | k ə ŋ ²¹² + k ɯ ²⁴ | 1 2 | k ə ŋ ²¹² + **k ɯ - ²⁴**

The disadvantage of the format is that alignments can now no longer be easily inspected from within text files, and it is almost impossible to create alignments without turning to the EDICTOR interface. 
However, it is clear that producing partial cognate sets with partial alignments in spreadsheet format would anyway be dangerous, since the human mind is not likely to master the annotation of partial cognates in any manual, unconstrained form, particularly well, without errors.

### 3.2 Editing Partial Cognates and Partial Alignments in EDICTOR 


In contrast to the edit operations for "normal" cognates in EDICTOR, partial cognates use a different design principle. Provided your data fulfils all format requirements, and you have your text-file with additional columns of TOKENS (with `+` as morpheme separator), COGIDS as the column to store partial cognates, and a column ALIGNMENT to store partial alignments (we don't distinguish the namespace between normal and partial alignments here, but it can be customized in the EDICTOR interface), you can start annotating partial cognates right away.

For this, load your file, for example our test file ```S09-edictor.tsv``` into EDICTOR. If you then select the column "COGIDS", and ideally just switch off columns like "COGID", you will see that the partial cognates have already been assigned in our example. A right mouse click will -- similarly too the function of it in the handling of "normal" cognates in the wordlist mode, open a pop-up window. But this time, the window does not lead directly to alignments, but instead shows from the perspective of the word that was clicked, how its morphemes find reflexes in the rest o fhe languages (and how these are aligned). In order to manually correct alignments at this point, the users can simply press on the little alignment symbol on top of each morpheme-cognate aset and edit the data accordingly.

![img](img/s9-edictor-alms.png)

A more straightforward way, however, is to use the specific partial cognate editor by clicking on EDIT->Partial Cognates. A (fake) popup-window will open and present the data concept by concept, basically similar to the way we know it from the normal cognate annotation tool. However, the difference is that we don't have two basic operations in this case, but instead a new annotation schema with which word morphemes can be assigned to cognate-sets in fashion reminding slightly of drag-and-drop systems. For our first item ("male dog"), this looks like teh following:

![img](img/s9-partial.png)

This view presents the cognate sets in a de-linearized fashion in which a column for each cognate set is added to a table of words. By clicking on the word inside the cognate set of a given cognate set table, the word will discarded from this column, and the token will be assigned the value `0`, indicating that the morpheme has not been assigned to any cognate set so far. The top-right column of each table always offers a new (fresh) cognate ID to place new words into that cannot be assigned to any related word. By clicking on several words in the TOKENS column, the user can *pre-select* them. When pressing the column header of a given column after preselecting a couple of morphemes, those morphemes will be placed into that column, and they can again be aligned directly by clicking on the alignment character placed on the right of each cognate set column header. EDICTOR will always make sure that the next ID provided has not been used before. The following screen video illustrates this procedure.

In [105]:
%%HTML
<video width="800" height="400" controls>
  <source src="img/s9-pcogs.mp4" type="video/mp4">
</video>

A final remark: if the column name for your partial cognates is different from "cogids", you will need to tell EDICTOR explicitly from the settings where the partial cognates are, similar to the way you can change the column name containing the full cognates.

## 4 Computing Partial Cognates with LingPy

We will work with a small dataset of Tujia languages, compiled originally by George Starostin and later modified to account for partial cognates by List et al. (2016). We have to change this dataset slightly, as it does not contain an explicit segmentation into morphemes. So we load the data first, manipulate it with help of LingPy, and then analyse it with help of the `Partial` class of LingPy. 

But let's first start loading the data.

In [106]:
from lingpy import *

wl = Wordlist('../data/S09-data.tsv')
print('Wordlist has {0} words, {1} meanings, and {2} varieties.'.format(len(wl), wl.height, wl.width))

Wordlist has 513 words, 109 meanings, and 5 varieties.


If we inspect only the first tokens for the concept "foot", we can see that the data is not yet morpheme-segmented, so we need to do this ourselves now from within LingPy (or manually, with help of EDICTOR).

In [107]:
firstp = wl.get_list(row="foot", flat=True)
for idx in firstp:
    print('{0:10}\t{1}'.format(wl[idx, 'doculect'], ' '.join(wl[idx, 'tokens'])))

Boluo_Tujia	a ³³ dʑ i ¹³
Dianfang_Tujia	tɕ i ²¹
Duogu_Tujia	tɕ i ²¹ pʰ a ²¹
Tanxi_Tujia	ʔ a ³³ dʑ i ³⁵
Tasha_Tujia	tɕ i ²¹


We do this with help of the function `syllabify`, which takes a list of tokens (or even a raw string) as input and infers the tokens, by 

1. checking whether tonal information can be found (in which case the method will assume that tones are at the end of each syllable and segment the data accordingly), or
2. using prosody as a proxy for syllabic segmentation 

Note that all in all, we are no dealing with true morpheme segmentation here, but rather prosody-based syllable-segmentation. But since in most SEA languages the syllable constitutes a morpheme, we can savely assume this for those datasets, but will need other means of morpheme segmentation prior to using LingPy's partial cognate detection when dealing with other languages from other language families.

Let us first look, how the function works:

In [108]:
from lingpy.sequence.sound_classes import syllabify
tkm = syllabify(['t', 'o', 'k', '⁵', 'p', 'a', 'n'], sep="+")
tkm2 = syllabify("tokmra", sep="!")
print(tkm)
print(tkm2)

['t', 'o', 'k', '⁵', '+', 'p', 'a', 'n']
['t', 'o', '!', 'k', 'm', 'r', 'a']


We can see that this nicely converts our sequence into our desired format. We can thus use this function to correct our Tujia data before passing it to the `Partial` algorithm.

In [109]:
for idx, tokens in wl.iter_rows('tokens'):
    wl[idx, 'tokens'] = syllabify(tokens)

And we can now finally start loading the data into the `Partial` class. We are lazy and simply pass it directly.
    

In [110]:
from lingpy.compare.partial import Partial
part = Partial(wl, segments='tokens')

Analyzing is as straightforward as with the `LexStat` class, with the difference being that we have a slightly modified command for partial cognate detection, as a `Partial` object is formally also a `LexStat` instance and could also be used for "normal" cognate detection.

In [111]:
part.partial_cluster(method='sca', cluster_method='upgma', threshold=0.45, ref='scaids')
for idx in firstp:
     print('{0:10}\t{1:25}\t{2}'.format(
         part[idx, 'doculect'], 
         ' '.join(part[idx, 'tokens']),
         ' '.join([str(x) for x in part[idx, 'scaids']])                                                                
     ))

Boluo_Tujia	a ³³ + dʑ i ¹³           	110 111
Dianfang_Tujia	tɕ i ²¹                  	111
Duogu_Tujia	tɕ i ²¹ + pʰ a ²¹        	111 114
Tanxi_Tujia	ʔ a ³³ + dʑ i ³⁵         	115 111
Tasha_Tujia	tɕ i ²¹                  	111


We can also apply the LexStat method here. For this purpose, we pre-calculate, as we also do it for the LexStat algorithm, the scorer for language-specific segment similarities and then apply the same procedure to cluster the data into cognate sets.

In [112]:
part.get_scorer(runs=1000)
part.partial_cluster(method='lexstat', cluster_method='upgma', threshold=0.55, ref='lexstatids')
for idx in firstp:
     print('{0:10}\t{1:25}\t{2:10}\t{3}'.format(
         part[idx, 'doculect'], 
         ' '.join(part[idx, 'tokens']),
         ' '.join([str(x) for x in part[idx, 'scaids']]),
         ' '.join([str(x) for x in part[idx, 'lexstatids']])  
     ))

2018-06-04 16:42:39,870 [WARNING] A different scoring function has already been calculated, overwriting previous settings.
                                                                                    

Boluo_Tujia	a ³³ + dʑ i ¹³           	110 111   	117 118
Dianfang_Tujia	tɕ i ²¹                  	111       	119
Duogu_Tujia	tɕ i ²¹ + pʰ a ²¹        	111 114   	119 121
Tanxi_Tujia	ʔ a ³³ + dʑ i ³⁵         	115 111   	117 118
Tasha_Tujia	tɕ i ²¹                  	111       	119


Last not least, we can even align the data for each partial cognate set. For this purpose, we can simply load the data into an alignment object, specify the column that stores the partial cognates, and align the data in the same way in which we learned this before. There is only one caveat when applying this procedure: LingPy usually tries to guess whether the input data is "fuzzy", i.e., whether the cognate sets are partial as opposed to full cognates. In order to do so, LingPy checks the type of the column storing the partial cognate sets. Within LingPy, some namespaces for partial cognates are pre-defined, for example "partialids", or "cogids". If users want to use other namespaces, they have to actively modify the type of the column in a given LingPy wordlist by iterating over each column cell and turning what LingPy reads in from the input data as a normal string (the default way to read the data of an unknown column format) and change them to lists of integers. 

To make sure that the data is really considered as "fuzzy" by LingPy, one should always check the `Alignments._mode` keyword and also explicitly pass the keyword `fuzzy=True` when loading alignments. Note, however, that assigning a "fuzzy" keyword only works if the data is in the correct format (list of integers), so unless you use standard names for partial cognates, like "cogids" or "partialids", you need to be very careful.

In [113]:
alms = Alignments(part, ref='lexstatids', fuzzy=True)
print("Loaded alignments, alignment mode is '{0}''.".format(alms._mode))

Loaded alignments, alignment mode is 'fuzzy''.


Aligning the data is then straightforward. In most cases, the simplest algorithm and the defaults can be used, as the alignment of morphemes is much simpler than the alignment of longer words. 

In [114]:
alms.align()
for idx in firstp:
     print('{0:10}\t{1:25}\t{2:10}\t{3}'.format(
         part[idx, 'doculect'], 
         ' '.join(part[idx, 'tokens']),
         ' '.join([str(x) for x in part[idx, 'lexstatids']]),
         ' '.join(alms[idx, 'alignment'])
     ))

Boluo_Tujia	a ³³ + dʑ i ¹³           	117 118   	- a ³³ + dʑ i ¹³
Dianfang_Tujia	tɕ i ²¹                  	119       	tɕ i ²¹
Duogu_Tujia	tɕ i ²¹ + pʰ a ²¹        	119 121   	tɕ i ²¹ + pʰ a ²¹
Tanxi_Tujia	ʔ a ³³ + dʑ i ³⁵         	117 118   	ʔ a ³³ + dʑ i ³⁵
Tasha_Tujia	tɕ i ²¹                  	119       	tɕ i ²¹


For reasons of consistency, we further recommend to change the "morpheme_separator" in LingPy's basic settings to "+" (currently not yet the default, but will be in future updates). 

In [115]:
rc(morpheme_separator="+")

If you rerun the code now, it will provide the "+" as a morpheme separator in the alignments, as expected.

In [116]:
alms = Alignments(part, ref='lexstatids', fuzzy=True)
alms.align()
for idx in firstp:
     print('{0:10}\t{1:25}\t{2:10}\t{3}'.format(
         part[idx, 'doculect'], 
         ' '.join(part[idx, 'tokens']),
         ' '.join([str(x) for x in part[idx, 'lexstatids']]),
         ' '.join(alms[idx, 'alignment'])
     ))

Boluo_Tujia	a ³³ + dʑ i ¹³           	117 118   	- a ³³ + dʑ i ¹³
Dianfang_Tujia	tɕ i ²¹                  	119       	tɕ i ²¹
Duogu_Tujia	tɕ i ²¹ + pʰ a ²¹        	119 121   	tɕ i ²¹ + pʰ a ²¹
Tanxi_Tujia	ʔ a ³³ + dʑ i ³⁵         	117 118   	ʔ a ³³ + dʑ i ³⁵
Tasha_Tujia	tɕ i ²¹                  	119       	tɕ i ²¹


An important note on internal representation of the alignments in LingPy: alignments are nested lists and the cognate set identifier for, say, the second morpheme in a given word is also the second cognate set identifier in the column providing the cognate identifiers for partial cognates. 

In LingPy's `Alignments` class, alignments are represented in a convenient form internally, given in the dictionary `Alignments.msa[REF]`, where `REF` is the name of the column (the `ref=COLUMN` attribute). The key of the dictionary is the cognate identifier, and the value is a dictionary providing information on the doculects, the alignment, and other aspects. If you access this value, you can use the `SCA` function to retrieve a `Multiple` object from a given alignment:

In [117]:
print(alms.msa['lexstatids'][1])

{'stamp': '# MSA\n# dataset    : lingpy-2018-06-04\n# collection : 1 ("I")\n# aligned by : LingPy Version 2.6.3 <www.lingpy.org>\n# created on : 2018-06-04 15:15\n# parameters : progressive_sca_-2_0.5_0.3_neighbor_0.5__T\n', '_sonority_consensus': [4, 7, 8], 'seqs': [['ŋ', 'o', '³³'], ['ŋ', 'a', '³⁵'], ['ŋ', 'a', '²⁴'], ['ŋ', 'o', '³³'], ['ŋ', 'a', '²⁴']], 'taxa': ['Boluo_Tujia', 'Dianfang_Tujia', 'Duogu_Tujia', 'Tanxi_Tujia', 'Tasha_Tujia'], 'seq_id': '1 ("I")', 'dataset': 'lingpy-2018-06-04', 'ID': [191, 190, 189, 192, 188], 'alignment': [['ŋ', 'o', '³³'], ['ŋ', 'a', '³⁵'], ['ŋ', 'a', '²⁴'], ['ŋ', 'o', '³³'], ['ŋ', 'a', '²⁴']], 'parameters': 'progressive_sca_-2_0.5_0.3_neighbor_0.5__T'}


This means, you can also pretty-print an alignment rather easily, as we can check for cognate set number 115 in our analysis:

In [118]:
msa = SCA(alms.msa['lexstatids'][115])
print(msa)

dʑ	i	³³
dz	i	³³


What you need to keep in mind, however, is that the alignment is showing only a part of the word, as it represents the alignment of the morphemes, not the word. This means, if you want to assemble parts of the data, and carry out certain checks with the partial alignment class in LingPy, you will need some tweaks to get it right. Specifically, if you want to determine, what part of an word a given alignment represents, you need to check for the position, which is currently not stored anywhere in the `Alignments` class.

In [119]:
msa = alms.msa['lexstatids'][115]
for i, idx in enumerate(msa['ID']):
    print('{0:10}  {1:10}  {2:25}  {3:10}  {4}'.format(
        alms[idx, 'doculect'],
        alms[idx, 'concept'],
        ' '.join(alms[idx, 'tokens']),
        ' '.join(msa['alignment'][0]),
        alms[idx, 'lexstatids'].index(115),
        
    ))

Boluo_Tujia  fly         m u ³³ + dʑ i ³³           dʑ i ³³     1
Tanxi_Tujia  fly         m u ³³ + dz i ³³           dʑ i ³³     1
